In [1]:
#CS221_11292018_0824 includes a RAKE class that applies RAKE on the input file
import tensorflow as tf
import pandas as pd
import json
import numpy as np
import sys
import collections


In [11]:
number_of_data_points_to_read= 10000 #Select number of data points to read
counter= 1
data=[]
path= '/Users/srishti/Desktop/reviews_Books_5.json'
with open(path) as f:
    for line in f:
        if counter <number_of_data_points_to_read:
            data.append(json.loads(line))
            counter+=1
        else: 
            break


In [12]:
# path= '/Users/srishti/Google Drive/000_7th Quarter/CS221/Project/untouched_data/reviews_Automotive_5.json'
# data = []
# with open(path) as f:
#     for line in f:
#         data.append(json.loads(line))

# d[1]
# for j in data[1]:
#     print j
# print data[1]['reviewText']

X=[] #reivews
Y=[] #ratings

for i in data:
    X.append(i['reviewText'])
    Y.append(i['overall'])

print X[1]
print Y[1]


This is one my must have books. It is a masterpiece of spirituality. I'll be the first to admit, its literary quality isn't much. It is rather simplistically written, but the message behind it is so powerful that you have to read it. It will take you to enlightenment.
5.0


In [13]:
asin_dict = collections.defaultdict(list)
for i in data:
    X.append(i['reviewText'])
    Y.append(i['overall'])
    if i['overall'] >= 4:
        asin_dict[i['asin']+"_pos"].append(i['reviewText'])
    else:
        asin_dict[i['asin']+"_neg"].append(i['reviewText'])

# Merges the list of reviews for each asin_pos and asin_neg into a single string
for j in asin_dict.keys():
	asin_dict[j] = " ".join(asin_dict[j])

In [14]:
# print asin_dict.keys()
# print asin_dict['0030707315_pos']
print len(asin_dict.keys())

320


In [15]:
#
embeddings_index = dict()
f = open('/Users/srishti/Google Drive/000_7th Quarter/CS221/Project/untouched_data/glove.6B/glove.6B.50d.txt')
for line in f:# first element of each line is the word and remaining elements are numerical represenation of each line 
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs# creates a dict of words and numerical representation of that word
f.close()

In [16]:
embeddings_index['dog']
embeddings_index['time']

array([ 0.02648  ,  0.33737  ,  0.065667 , -0.11609  ,  0.41651  ,
       -0.21142  , -0.69582  ,  0.2822   , -0.36077  , -0.13822  ,
        0.012094 ,  0.086227 , -0.84638  ,  0.057195 ,  1.1582   ,
        0.14703  , -0.0049197, -0.24899  , -0.96014  , -0.3038   ,
        0.23972  ,  0.21058  ,  0.40608  ,  0.17789  ,  0.55253  ,
       -1.6357   , -0.17784  , -0.45222  ,  0.45805  ,  0.14239  ,
        3.7087   ,  0.40289  , -0.4083   , -0.29304  ,  0.030857 ,
       -0.15361  ,  0.10607  ,  0.63397  ,  0.12397  , -0.25349  ,
       -0.10344  ,  0.0069768, -0.17328  ,  0.35536  , -0.46369  ,
        0.15285  ,  0.41475  , -0.3398   , -0.23043  ,  0.19069  ],
      dtype=float32)

In [17]:
# -*- coding: utf-8 -*-
"""Implementation of Rapid Automatic Keyword Extraction algorithm.
As described in the paper `Automatic keyword extraction from individual
documents` by Stuart Rose, Dave Engel, Nick Cramer and Wendy Cowley.
"""

import string
from collections import Counter, defaultdict
from itertools import chain, groupby, product

import nltk
from enum import Enum
from nltk.tokenize import wordpunct_tokenize


class Metric(Enum):
    """Different metrics that can be used for ranking."""

    DEGREE_TO_FREQUENCY_RATIO = 0  # Uses d(w)/f(w) as the metric
    WORD_DEGREE = 1  # Uses d(w) alone as the metric
    WORD_FREQUENCY = 2  # Uses f(w) alone as the metric


class Rake(object):
    """Rapid Automatic Keyword Extraction Algorithm."""

    def __init__(
        self,
        stopwords=None,
        punctuations=None,
        language="english",
        ranking_metric=Metric.DEGREE_TO_FREQUENCY_RATIO,
        max_length=100000,
        min_length=1,
    ):
        """Constructor.
        :param stopwords: List of Words to be ignored for keyword extraction.
        :param punctuations: Punctuations to be ignored for keyword extraction.
        :param language: Language to be used for stopwords
        :param max_length: Maximum limit on the number of words in a phrase
                           (Inclusive. Defaults to 100000)
        :param min_length: Minimum limit on the number of words in a phrase
                           (Inclusive. Defaults to 1)
        """
        # By default use degree to frequency ratio as the metric.
        if isinstance(ranking_metric, Metric):
            self.metric = ranking_metric
        else:
            self.metric = Metric.DEGREE_TO_FREQUENCY_RATIO

        # If stopwords not provided we use language stopwords by default.
        self.stopwords = stopwords
        if self.stopwords is None:
            self.stopwords = nltk.corpus.stopwords.words(language)

        # If punctuations are not provided we ignore all punctuation symbols.
        self.punctuations = punctuations
        if self.punctuations is None:
            self.punctuations = string.punctuation

        # All things which act as sentence breaks during keyword extraction.
        self.to_ignore = set(chain(self.stopwords, self.punctuations))

        # Assign min or max length to the attributes
        self.min_length = min_length
        self.max_length = max_length

        # Stuff to be extracted from the provided text.
        self.frequency_dist = None
        self.degree = None
        self.rank_list = None
        self.ranked_phrases = None

    def extract_keywords_from_text(self, text):
        """Method to extract keywords from the text provided.
        :param text: Text to extract keywords from, provided as a string.
        """
        sentences = nltk.tokenize.sent_tokenize(text)
        self.extract_keywords_from_sentences(sentences)

    def extract_keywords_from_sentences(self, sentences):
        """Method to extract keywords from the list of sentences provided.
        :param sentences: Text to extraxt keywords from, provided as a list
                          of strings, where each string is a sentence.
        """
        phrase_list = self._generate_phrases(sentences)
        self._build_frequency_dist(phrase_list)
        self._build_word_co_occurance_graph(phrase_list)
        self._build_ranklist(phrase_list)

    def get_ranked_phrases(self):
        """Method to fetch ranked keyword strings.
        :return: List of strings where each string represents an extracted
                 keyword string.
        """
        return self.ranked_phrases

    def get_ranked_phrases_with_scores(self):
        """Method to fetch ranked keyword strings along with their scores.
        :return: List of tuples where each tuple is formed of an extracted
                 keyword string and its score. Ex: (5.68, 'Four Scoures')
        """
        return self.rank_list

    def get_word_frequency_distribution(self):
        """Method to fetch the word frequency distribution in the given text.
        :return: Dictionary (defaultdict) of the format `word -> frequency`.
        """
        return self.frequency_dist

    def get_word_degrees(self):
        """Method to fetch the degree of words in the given text. Degree can be
        defined as sum of co-occurances of the word with other words in the
        given text.
        :return: Dictionary (defaultdict) of the format `word -> degree`.
        """
        return self.degree

    def _build_frequency_dist(self, phrase_list):
        """Builds frequency distribution of the words in the given body of text.
        :param phrase_list: List of List of strings where each sublist is a
                            collection of words which form a contender phrase.
        """
        self.frequency_dist = Counter(chain.from_iterable(phrase_list))

    def _build_word_co_occurance_graph(self, phrase_list):
        """Builds the co-occurance graph of words in the given body of text to
        compute degree of each word.
        :param phrase_list: List of List of strings where each sublist is a
                            collection of words which form a contender phrase.
        """
        co_occurance_graph = defaultdict(lambda: defaultdict(lambda: 0))
        for phrase in phrase_list:
            # For each phrase in the phrase list, count co-occurances of the
            # word with other words in the phrase.
            #
            # Note: Keep the co-occurances graph as is, to help facilitate its
            # use in other creative ways if required later.
            for (word, coword) in product(phrase, phrase):
                co_occurance_graph[word][coword] += 1
        self.degree = defaultdict(lambda: 0)
        for key in co_occurance_graph:
            self.degree[key] = sum(co_occurance_graph[key].values())

    def _build_ranklist(self, phrase_list):
        """Method to rank each contender phrase using the formula
              phrase_score = sum of scores of words in the phrase.
              word_score = d(w)/f(w) where d is degree and f is frequency.
        :param phrase_list: List of List of strings where each sublist is a
                            collection of words which form a contender phrase.
        """
        self.rank_list = []
        for phrase in phrase_list:
            rank = 0.0
            for word in phrase:
                if self.metric == Metric.DEGREE_TO_FREQUENCY_RATIO:
                    rank += 1.0 * self.degree[word] / self.frequency_dist[word]
                elif self.metric == Metric.WORD_DEGREE:
                    rank += 1.0 * self.degree[word]
                else:
                    rank += 1.0 * self.frequency_dist[word]
            self.rank_list.append((rank, " ".join(phrase)))
        self.rank_list.sort(reverse=True)
        self.ranked_phrases = [ph[1] for ph in self.rank_list]

    def _generate_phrases(self, sentences):
        """Method to generate contender phrases given the sentences of the text
        document.
        :param sentences: List of strings where each string represents a
                          sentence which forms the text.
        :return: Set of string tuples where each tuple is a collection
                 of words forming a contender phrase.
        """
        phrase_list = set()
        # Create contender phrases from sentences.
        for sentence in sentences:
            word_list = [word.lower() for word in wordpunct_tokenize(sentence)]
            phrase_list.update(self._get_phrase_list_from_words(word_list))
        return phrase_list

    def _get_phrase_list_from_words(self, word_list):
        """Method to create contender phrases from the list of words that form
        a sentence by dropping stopwords and punctuations and grouping the left
        words into phrases. Only phrases in the given length range (both limits
        inclusive) would be considered to build co-occurrence matrix. Ex:
        Sentence: Red apples, are good in flavour.
        List of words: ['red', 'apples', ",", 'are', 'good', 'in', 'flavour']
        List after dropping punctuations and stopwords.
        List of words: ['red', 'apples', *, *, good, *, 'flavour']
        List of phrases: [('red', 'apples'), ('good',), ('flavour',)]
        List of phrases with a correct length:
        For the range [1, 2]: [('red', 'apples'), ('good',), ('flavour',)]
        For the range [1, 1]: [('good',), ('flavour',)]
        For the range [2, 2]: [('red', 'apples')]
        :param word_list: List of words which form a sentence when joined in
                          the same order.
        :return: List of contender phrases that are formed after dropping
                 stopwords and punctuations.
        """
        groups = groupby(word_list, lambda x: x not in self.to_ignore)
        phrases = [tuple(group[1]) for group in groups if group[0]]
        return list(
            filter(
                lambda x: self.min_length <= len(x) <= self.max_length, phrases
            )
         )

In [18]:
# rl_list=[]
counter=0
rl_dict={}
for i in asin_dict.keys():
    if counter < 2:
#         print asin_dict[i]
        c = Rake()
        c.extract_keywords_from_text(asin_dict[i])
        rl_dict[i]= c.rank_list
        counter+=1
    else:
        break
#     rl_list.append(c.rank_list)

In [19]:
# d = Rake()
# d.extract_keywords_from_text('I love cats and cats')
# print d.rank_list
# print d.ranked_phrases
print rl_dict

{u'000100039X_pos': [(46.75555555555555, u'jonathan livingston seagull &# 34 ;, deeply thought'), (44.17058823529412, u'strange land &# 34 ;, semantically precise yet'), (33.28333333333333, u'early 20th century critic claude bragdon wrote'), (31.81416666666667, u'quite awhile ago ... sigh .... 16'), (30.2, u'student slackers paradise book report choice'), (28.161064425770306, u'nice yet inexpensive gift containing valuable lessons'), (26.323809523809523, u'unique prose poetry rythym flows like wine'), (25.072727272727274, u'high school graduation present many years ago'), (23.333333333333336, u'shopping cart sit largely unread'), (23.0, u'border guard recognises lao tzu'), (23.0, u'adult responsibilities loom ever taller'), (22.50392156862745, u'yet extremely penetrating like &# 34'), (21.076923076923077, u'dad passed away twenty years ago'), (20.660256410256412, u'comfy girlhood world slips away'), (20.61111111111111, u'sister mary olga fortitude ,"'), (20.414545454545454, u'johann se

In [ ]:
#implementing K-means
#Use the most common thing on the collated reviews 
#use the most common words vector to create a vector of embeddings by summing the multiplication of occurance to the embedding value of the word or phrase
#

In [52]:
#converts reviews to embedding dict where key is the ASIN of a book
count=0
review_dict={}
number_of_reviews_for_clustering=5#select how many reviews are being used for clustering, small number reduces computation time. 
max_word_per_review= 10#model parameter
lower_cutoff_for_word_to_be_considered=1#model parameter
for i in rl_dict.keys():
    print i
    if count< number_of_reviews_for_clustering :
        count +=1
#         print rl_dict[i]
    #     print rl_list[i][0]
        count_of_words_used=0
        for j in rl_dict[i]:
            if count_of_words_used < max_word_per_review:
                count_of_words_used+=1
                if j[0] > lower_cutoff_for_word_to_be_considered:
                    if j[1] not in embeddings_index.keys():
                        for k in j[1].split():
            #                 print ("k=" + k)
                            if k in embeddings_index.keys():
                                if i not in review_dict.keys():

                                    review_dict[i]= j[0]* embeddings_index[k] 
                                else:
                                    review_dict[i]+= j[0]* embeddings_index[k] 
    #                         else:
    #                             print k
                    else:
                        if j[1] in embeddings_index.keys():
            #                 print ("j=" + j[1])
                #             review_dict[i]+= rl_list[j][0]* embeddings_index[j][1]
                            if i not in review_dict.keys():
                                review_dict[i]= j[0]* embeddings_index[j[1]]
                            else:
                                review_dict[i]+= j[0]* embeddings_index[j[1]]
    #                     else:
    #                         print j[1]
            
            





000100039X_pos
0002154463_neg


/Users/srishti/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:17: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
/Users/srishti/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:20: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [55]:
print review_dict

{u'000100039X_pos': array([ 2.7731717e+02,  4.0619586e+02, -3.8585797e+02, -5.8682123e+02,
        6.5531268e+02,  6.8793243e+01, -8.4993469e+02, -2.3294962e+02,
       -2.5476700e+02,  2.9207776e+02, -3.6567105e+02,  1.5591853e+02,
       -2.7069012e+01, -1.5147432e+02,  7.3841382e+02, -1.1459206e+02,
        2.5618869e+02, -1.8670209e+02, -3.0384470e+02, -1.7710843e+02,
       -7.1478088e+01,  4.2705069e+02,  3.6503476e+02,  6.3009670e+01,
        5.7988843e+02, -1.7120302e+03, -8.1559479e+02,  1.0510909e+02,
        3.0434308e+02, -1.5469287e+02,  3.9525046e+03, -3.4181036e+02,
        1.0505040e+02, -4.0130011e+02,  1.9406223e+02, -3.6493182e+00,
       -5.5965591e+01,  4.8521457e+02,  9.3856110e+01, -1.2499247e+02,
       -8.8700378e+01,  1.2667018e+02,  1.9505418e+02,  2.8177405e+02,
       -2.0112576e+02,  3.9512274e+02, -1.5621758e+02,  7.9334259e+00,
       -1.6734084e+02,  1.2741069e+02], dtype=float32), u'0002154463_neg': array([ 6.62336884e+01,  2.32493439e+01, -1.12529701e

In [54]:
print rl_dict['0002154463_neg']

[(9.0, u'vegetarians might go'), (9.0, u'sugar cane came'), (9.0, u'steamed asparagus topped'), (9.0, u'several silly errors'), (8.5, u'full page pictures'), (8.25, u'italian scenery paired'), (8.25, u'classical italian cooking'), (8.0, u'lorenza de medici'), (5.0, u'de medici'), (4.5, u'gorgeous pictures'), (4.25, u'italian edition'), (4.0, u'using hazzan'), (4.0, u'severely dissapointed'), (4.0, u'seldom seen'), (4.0, u'rather blah'), (4.0, u'peasant kitchens'), (4.0, u'one left'), (4.0, u'new world'), (4.0, u'marcella hazzan'), (4.0, u'greatly prefer'), (4.0, u'gorgeous picture'), (4.0, u'food illustrations'), (4.0, u'fine dish'), (4.0, u'dull enough'), (4.0, u'croque monsieur'), (4.0, u'50 dollars'), (3.5, u'unwieldy books'), (3.5, u'much better'), (3.5, u'mexico book'), (3.5, u'mediocre recipes'), (3.5, u'collect cookbooks'), (3.5, u'beautiful cookbook'), (2.25, u'italian'), (1.5, u'recipes'), (1.5, u'mexico'), (1.5, u'cookbooks'), (1.5, u'books'), (1.5, u'better'), (1.5, u'beauti